# Importações e configurações

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc

#import matplotlib.pyplot as plt

import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
RANDOM_SEED = 42

# Carregando os dados

In [4]:
df = pd.read_excel("../data/cleaned_datas/cleaned_data.xlsx")

In [5]:
df

,abastecimento de água_fonte/água não tratada,abastecimento de água_outros,abastecimento de água_poço/água não tratada,abastecimento de água_rede geral/água tratada,alimentação no ambiente escolar,aluno contemplado com bolsa?,como você acessa os serviços de saúde_plano de saúde (privado),como você acessa os serviços de saúde_público e privado,como você acessa os serviços de saúde_sistema público de saúde,como você acessa/acessou à educação básica (ensino médio)_bolsista de escola privada,...,sexo_feminino,sexo_masculino,turno_integral,turno_manhã,turno_noite,turno_tarde,você possui filhos entre 0 e 6 anos,você possui filhos maiores que 6 anos,você é chefe de família ou responsável pela própria subsistência?,"é cotista por renda inferior a 1,5 salário mínimo?"
0,False,False,True,False,70.0,False,False,False,True,False,...,True,False,False,True,False,False,0,0,False,False
1,False,False,False,True,25.0,True,False,False,True,False,...,True,False,False,False,False,True,0,0,False,False
2,False,False,True,False,180.0,True,False,False,True,False,...,True,False,False,True,False,False,0,0,False,True
3,False,False,False,True,150.0,False,False,False,True,False,...,True,False,False,True,False,False,0,0,False,False
4,False,False,False,True,50.0,True,False,False,True,False,...,False,True,True,False,False,False,0,0,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14533,False,False,False,True,45.0,True,True,False,False,False,...,False,True,False,False,False,True,0,0,False,False
14534,False,False,False,True,0.0,True,False,False,True,False,...,True,False,False,False,False,True,0,0,False,True
14535,False,True,False,False,10.0,True,False,False,True,False,...,True,False,False,False,True,False,0,2,True,True
14536,False,False,False,True,40.0,True,False,False,True,False,...,False,True,False,False,True,False,0,0,True,True


# Pré-processamento dos dados

## Dropando a coluna Relatos de vida (TEMPORÁRIO)

In [6]:
df = df.drop(columns=["relato de vida"])

## Transformando float64 em float

In [7]:
df.dtypes

abastecimento de água_fonte/água não tratada                            bool
abastecimento de água_outros                                            bool
abastecimento de água_poço/água não tratada                             bool
abastecimento de água_rede geral/água tratada                           bool
alimentação no ambiente escolar                                      float64
                                                                      ...   
turno_tarde                                                             bool
você possui filhos entre 0 e 6 anos                                    int64
você possui filhos maiores que 6 anos                                  int64
você é chefe de família ou responsável pela própria subsistência?       bool
é cotista por renda inferior a 1,5 salário mínimo?                      bool
Length: 106, dtype: object

## Analisando correlação entre features

Por enquanto, vou manter sem o drop destas colunas correlacionadas.

In [8]:
"""
def get_highly_correlated_features(correlation_matrix, threshold):
  correlated_pairs = []
  for i in range(len(correlation_matrix.columns)):
    for j in range(i):
      if abs(correlation_matrix.iloc[i, j]) > threshold:
        pair = (correlation_matrix.columns[i], correlation_matrix.columns[j])
        coefficient = correlation_matrix.iloc[i, j]
        correlated_pairs.append((pair, coefficient))
  return sorted(correlated_pairs, key= lambda pair: pair[1], reverse=True)
"""

'\ndef get_highly_correlated_features(correlation_matrix, threshold):\n  correlated_pairs = []\n  for i in range(len(correlation_matrix.columns)):\n    for j in range(i):\n      if abs(correlation_matrix.iloc[i, j]) > threshold:\n        pair = (correlation_matrix.columns[i], correlation_matrix.columns[j])\n        coefficient = correlation_matrix.iloc[i, j]\n        correlated_pairs.append((pair, coefficient))\n  return sorted(correlated_pairs, key= lambda pair: pair[1], reverse=True)\n'

In [9]:
"""
corr_matrix = df.corr().abs()
correlation_list = get_highly_correlated_features(corr_matrix, 0.95)
"""

'\ncorr_matrix = df.corr().abs()\ncorrelation_list = get_highly_correlated_features(corr_matrix, 0.95)\n'

In [10]:
#correlation_list[:10]

In [11]:
"""
f2drop = []
for feature_pair, _ in correlation_list:
  if feature_pair[0] not in f2drop and feature_pair[1] not in f2drop:
    f2drop.append(feature_pair[1])
"""

'\nf2drop = []\nfor feature_pair, _ in correlation_list:\n  if feature_pair[0] not in f2drop and feature_pair[1] not in f2drop:\n    f2drop.append(feature_pair[1])\n'

In [12]:
#f2drop

In [13]:
#df = df.drop(f2drop, axis='columns')

## Dividindo dados nos conjuntos de treino, validação e teste

In [14]:
labels = df[['aluno contemplado com bolsa?']].copy()
df = df.drop(columns=['aluno contemplado com bolsa?'])

In [15]:
X_train, X_test, y_train, y_test = train_test_split(df, labels, test_size=0.1, random_state=RANDOM_SEED)

## Normalizando os dados numéricos

In [16]:
#numeric_columns = df.select_dtypes(include=['number']).columns

std_scaler = StandardScaler()
std_scaler = std_scaler.fit(X_train)

X_train = std_scaler.transform(X_train)
X_test = std_scaler.transform(X_test)

# Inteligências artificiais

## Naive Bayesian

### Treinamento

In [17]:
model = GaussianNB()

In [18]:
model.fit(X_train, y_train)

GaussianNB()

### Teste

In [19]:
predicts = model.predict(X_test)

In [20]:
probs = model.predict_proba(X_test)

In [21]:
class_labels = model.classes_
print(class_labels)

[False  True]


In [22]:
def get_overall_metrics(y_true, y_pred):
  tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
  acc = (tp+tn)/(tp+tn+fp+fn)
  tpr = tp/(tp+fn)
  fpr = fp/(fp+tn)
  precision = tp/(tp+fp)
  f1 = (2*tpr*precision)/(tpr+precision)
  return {'acc':acc,'tpr':tpr,'fpr':fpr,'precision':precision,'f1-score':f1}

In [23]:
get_overall_metrics(y_test, predicts)

{'acc': np.float64(0.7386519944979367),
 'tpr': np.float64(0.7568181818181818),
 'fpr': np.float64(0.289198606271777),
 'precision': np.float64(0.8004807692307693),
 'f1-score': np.float64(0.7780373831775702)}